## Okex API exploration - Market data

In [37]:
from okex import *
import pandas as pd
from datetime import datetime, timedelta
import plotly.graph_objects as go 

- API key, secret and passphrase should be modified in the config.py file (in okex module)  
- My trading bot in Okex is currently being used.

### III. Market data

#### 1. Instanciate Market API object

In [2]:
market = MarketAPI(API_KEY, API_SECRET, PASSPHRASE, flag='1')

#### 2. Get all available currencies tickers (price + volume)

In [14]:
def get_tickers(market, instType='SWAP'):
    tickers = market.get_tickers(instType='SWAP')
    if len(tickers['data'])==0:
        return pd.DataFrame()
    df = pd.DataFrame([x.values() for x in tickers['data']], columns = tickers['data'][0].keys())
    df['ts'] = df['ts'].astype('int64').map(lambda x: pd.to_datetime(x, unit='ms'))
    return df

In [66]:
get_tickers(market)

,instType,instId,last,lastSz,askPx,askSz,bidPx,bidSz,open24h,high24h,low24h,volCcy24h,vol24h,ts,sodUtc0,sodUtc8
0,SWAP,ETC-USDT-SWAP,34.18,20,34.19,220,34.177,216,32.661,34.698,32.251,74541500,7454150,2022-08-23 22:06:20.409,34.066,33.895
1,SWAP,SUSHI-USDT-SWAP,1.215,52386,1.216,62348,1.215,10226,1.163,1.223,1.152,97877096,97877096,2022-08-23 22:06:14.112,1.189,1.203
2,SWAP,XLM-USDT-SWAP,0.11047,0,0.11059,4841,0.11052,5776,0.10797,0.11105,0.1072,1470150000,14701500,2022-08-23 22:06:18.512,0.10949,0.10964
3,SWAP,DASH-USDT-SWAP,48.42,666,48.47,14017,48.43,13258,46.36,48.5,45.61,3674593.8,36745938,2022-08-23 22:06:18.612,47.24,47.94
4,SWAP,THETA-USDT-SWAP,1.234,4150,1.2352,22,1.234,4902,1.191,1.239,1.171,75696700,7569670,2022-08-23 22:06:20.113,1.217,1.224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,SWAP,BNT-USDT-SWAP,0.4924,27652,0.4923,137016,0.4917,160075,0.4812,0.4946,0.4714,13271150,13271150,2022-08-23 22:06:20.030,0.4898,0.4887
97,SWAP,BAL-USDT-SWAP,6.579,78618,6.622,87266,6.617,91357,6.109,6.705,6.083,25650624.4,256506244,2022-08-23 22:06:20.912,6.268,6.554
98,SWAP,SRM-USDT-SWAP,0.859,58713,0.861,89104,0.858,59703,0.81,0.862,0.8,30634083,30634083,2022-08-23 22:06:20.813,0.826,0.847
99,SWAP,AXS-USDT-SWAP,14.72,2915,14.73,46614,14.71,34512,14.02,14.86,13.7,11624507.1,116245071,2022-08-23 22:06:19.027,14.25,14.43


#### 3. Get a specific currency ticker

In [11]:
def get_ticker(market, instId):
    tickers = market.get_ticker(instId)
    if len(tickers['data'])==0:
        return pd.DataFrame()
    df = pd.DataFrame([x.values() for x in tickers['data']], columns = tickers['data'][0].keys())
    df['ts'] = df['ts'].astype('int64').map(lambda x: pd.to_datetime(x, unit='ms'))
    return df

In [12]:
get_ticker(market, 'ETH-USDT-SWAP')

,instType,instId,last,lastSz,askPx,askSz,bidPx,bidSz,open24h,high24h,low24h,volCcy24h,vol24h,ts,sodUtc0,sodUtc8
0,SWAP,ETH-USDT-SWAP,1644.82,3331,1644.9,10,1644.81,4640,1575.82,1700,1542.17,7384952.83,738495283,2022-08-23 21:26:04.810,1624.75,1640.22


#### 4. Get candlesticks

In [230]:
def get_candlesticks(market, instId, after=None, before=None, limit=300, bar='15m'):
    candles = market.get_index_candlesticks('ETH-USDT-SWAP', after=after, before=before, limit=limit, bar= bar)
    if len(candles['data'])==0:
        return pd.DataFrame()
    df = pd.DataFrame(candles['data'], columns = ['ts', 'o', 'h', 'l', 'c', 'vol', 'volCcy'])
    df['ts'] = df['ts'].astype('int64').map(lambda x: pd.to_datetime(x, unit='ms'))
    return df

In [231]:
df = get_candlesticks(market, 'ETH-USDT-SWAP')

In [76]:
def graph_candlestick(df): 
    a = go.Candlestick(x=df['ts'],
                   high=df['h'],
                   low=df['l'],
                   open=df['o'],
                   close=df['c'])
    figure = go.Figure(f)
    figure.show()

In [77]:
graph_candlestick(df)

#### 5. Get candlesticks history

- Candlestick history reading using Okex API is limited by 100 data points per request. That is why, we will be proceeding by batches of 100.  
- In order to specify period, we are using the date parameters: after & before
- The unit period is specified using the parameter "bar" which takes the values ("1m"(by default), "3m", ..., "15m", ... "4H", ...) 

In [308]:
def get_history(market, instId, after, before, bar="1D"):
    candles = market.get_candlesticks('BTC-USDT', bar=bar, before=before, after=after)
    if len(candles['data'])==0:
        return pd.DataFrame()
    df = pd.DataFrame(candles['data'], columns = ['ts', 'o', 'h', 'l', 'c', 'vol', 'volCcy'])
    df['ts'] = df['ts'].astype('int64').map(lambda x: pd.to_datetime(x, unit='ms'))
    return df 

In [ ]:
# Testing with constant parameters
instId = "BTC-USDT-SWAP"
get_history(market, instId, after="1 Jan, 2021", before=None)

Okex API doesn't give historical data with specific dates. After & Before parameters aren't working, I tested all timestamp and date formats without success